### Solving Sparse Regression using `NExOS.jl`

The sparse regression problem (also known as regressor selection problem) is concerned with approximating a vector $b\in\mathbf{R}^{m}$ with a linear combination of at most $k$ columns of a matrix $A\in\mathbf{R}^{m\times d}$ with bounded coefficients. The problem can be written as the following optimization
problem
$$
\begin{equation}
\begin{array}{ll}
\textrm{minimize} & \|Ax-b\|_{2}^{2}+\frac{\beta}{2}\|x\|^{2}\\
\textrm{subject to} & \mathbf{card}(x)\leq k\\
 & \|x\|_{\infty}\leq M,
\end{array}
\end{equation}
$$
where $x\in\mathbf{R}^{d}$ is the decision variable, and $A\in\mathbf{R}^{m\times d},b\in\mathbf{R}^{m},$ and $M>0$ are problem data.

First, load the packages.

In [1]:
using Random, NExOS, ProximalOperators

Let us generate some random data for this problem.

In [2]:
m = 25
n = 50
A = randn(m,n)
A = randn(m,n)
b = randn(m)
M = 100
k = convert(Int64, round(m/3))
beta = 10^-10

1.0000000000000006e-10

Create the problem instance in `NExOS`.

In [3]:
C = SparseSet(M, k) # Create the set
f = LeastSquares(A, b, iterative = true) # Create the function
settings = Settings(μ_max = 2, μ_min = 1e-8, μ_mult_fact = 0.85, verbose = false, freq = 250, γ_updt_rule = :adaptive, β = beta) # settings
z0 = zeros(n) # create an initial point
problem = Problem(f, C, settings.β, z0) # problem instance

Problem{ProximalOperators.LeastSquaresIterative{1,Float64,Float64,Array{Float64,2},Array{Float64,1},ProximalOperators.AAc},SparseSet{Int64,Int64},Float64,Array{Float64,1}}(description : Least squares penalty
domain      : n/a
expression  : n/a
parameters  : n/a, SparseSet{Int64,Int64}(100, 8), 1.0000000000000006e-10, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

Time to solve the problem.

In [4]:
state_final = solve!(problem, settings)

State{Array{Float64,1},Int64,Float64}([-1.2524621301508779e-8, -3.6751270002122006e-9, -0.39320187373700755, 9.22653516573817e-9, 1.5129706915674935e-8, 0.4357960444260704, -2.259401393269908e-8, 1.3889241406124104e-9, 2.9730535810395402e-9, 3.06400647086749e-9  …  -3.2479118083545196e-8, 4.836971965759645e-9, 9.191892814264385e-9, 6.675726671290362e-9, -5.168506057262721e-9, -2.2741186563059717e-8, -0.33056126538230574, 2.2996069601239075e-8, -1.5840142859842104e-8, 0.29452540272689054], [-1.3281798144261228e-8, -3.897335289126129e-9, -0.39320187480552615, 9.784345340772926e-9, 1.604439773046142e-8, 0.4357960434548361, -2.3960007157182014e-8, 1.4729021137531158e-9, 3.1527745420703963e-9, 3.2492558819354536e-9  …  -3.4442740642721185e-8, 5.129401262652161e-9, 9.747596517998287e-9, 7.079381206637663e-9, -5.480975565878605e-9, -2.411606745594044e-8, -0.33056126444915457, 2.4386342208835653e-8, -1.6797792548192827e-8, 0.2945254029210682], [1.2457152923814998e-7, 3.655358749296986e-8, -0.3

Let us take a look at the quality of the solution.

In [5]:
log10(state_final.fxd_pnt_gap) <= -4 # if the fixed point gap is less than 10^-4 (to determin if the algorithm has converged)

true

In [6]:
log10(state_final.fsblt_gap) <= -4 # this is to test if the found solution by NExOS is locally optimal

true

In [7]:
f(state_final.x) # this gives the objective value of the solution found by NExOS

1.5220193349339144